In [1]:
import pymysql
import requests

### 下載分析pm2.5 opendata資料 ###

In [5]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON"
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [6]:
resp=requests.get(url,verify=False)
resp

c:\Users\USER\Desktop\django\pm25_MYSQL\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [7]:
datas=resp.json()['records']
datas

[{'site': '員林',
  'county': '彰化縣',
  'pm25': '5',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '4',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '3',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '3',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '10',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '3',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '17',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '2',
  'datacreationdate'

In [8]:
len(datas)

1000

In [9]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '5',
 'datacreationdate': '2025-08-20 20:00',
 'itemunit': 'μg/m3'}

### 建立資料表

In [10]:
table_str="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)
"""
print(table_str)


create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)



### 連接資料庫

In [2]:
conn=pymysql.connect(
host="mysql-340eb03b-yulun9559-3531.i.aivencloud.com",
user="avnadmin",
password="AVNS_FEVJdhCBCSlMbUCUjR4",
port=19936,
database="defaultdb"
)

cursor=conn.cursor()
cursor

In [5]:
cursor.execute(table_str)
conn.commit()

NameError: name 'cursor' is not defined

### 取得單一縣市的站點數值

In [9]:
county="臺北市"

#取得最新資料
sqlstr='''
select site,pm25,datacreationdate from pm25
where county=%s
and datacreationdate=(select max(datacreationdate) from pm25);
'''
cursor.execute(sqlstr,(county,))
datas=cursor.fetchall()
datas

(('陽明', 14, datetime.datetime(2025, 9, 3, 18, 0)),
 ('大同', 12, datetime.datetime(2025, 9, 3, 18, 0)),
 ('松山', 10, datetime.datetime(2025, 9, 3, 18, 0)),
 ('古亭', 10, datetime.datetime(2025, 9, 3, 18, 0)),
 ('萬華', 8, datetime.datetime(2025, 9, 3, 18, 0)),
 ('中山', 10, datetime.datetime(2025, 9, 3, 18, 0)),
 ('士林', 12, datetime.datetime(2025, 9, 3, 18, 0)))

### 寫入一筆資料

In [13]:
sqlstr="insert into pm25(site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}','{}','{}','{}')"
data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])

['員林', '彰化縣', '5', '2025-08-20 20:00', 'μg/m3']


"insert into pm25(site,county,pm25,datacreationdate,itemunit)      values('員林','彰化縣','5','2025-08-20 20:00','μg/m3')"

In [14]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

### 一次寫入
- 加上 ignore

In [ ]:
sqlstr="insert ignore into pm25(site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"

# 移除pm25空字串部分
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]

cursor.executemany(sqlstr,values)
conn.commit()

In [35]:
conn.close()

### 取得各縣市站點的平均值

In [7]:
cursor.execute("select avg(pm25) from pm25;")
float(cursor.fetchone()[0])

7.6674

In [11]:
sqlstr="""
select county,round(avg(pm25),2) from pm25 group by county;
"""
cursor.execute(sqlstr)
cursor.fetchall()

(('彰化縣', Decimal('11.34')),
 ('新北市', Decimal('4.00')),
 ('雲林縣', Decimal('10.35')),
 ('臺東縣', Decimal('2.51')),
 ('澎湖縣', Decimal('7.73')),
 ('金門縣', Decimal('10.26')),
 ('南投縣', Decimal('10.06')),
 ('高雄市', Decimal('10.42')),
 ('桃園市', Decimal('5.64')),
 ('宜蘭縣', Decimal('3.17')),
 ('臺北市', Decimal('4.29')),
 ('花蓮縣', Decimal('2.59')),
 ('屏東縣', Decimal('8.54')),
 ('臺南市', Decimal('8.76')),
 ('嘉義市', Decimal('7.26')),
 ('嘉義縣', Decimal('9.29')),
 ('臺中市', Decimal('11.37')),
 ('苗栗縣', Decimal('10.13')),
 ('新竹市', Decimal('9.41')),
 ('新竹縣', Decimal('8.63')),
 ('基隆市', Decimal('2.28')),
 ('連江縣', Decimal('6.50')))

In [12]:
conn.close()

In [29]:
import pandas as pd

In [ ]:
cursor.execute("select * from pm25")

datas=cursor.fetchall()
datas